<a href="https://colab.research.google.com/github/HST0077/Financial-Engineering-Python/blob/main/Quantlib00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **대한민국 국채 Yield 가져오기**

In [ ]:
!pip install selenium webdriver_manager QuantLib
!apt-get update
!apt install chromium-chromedriver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [773 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,356 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,898 kB]
Hit:13 https://ppa.launchpadc

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
from io import StringIO
import datetime
import QuantLib as ql
from dateutil.relativedelta import relativedelta


options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')

driver = webdriver.Chrome(options=options)

url = 'https://www.kofiabond.or.kr/websquare/websquare.html?w2xPath=/xml/bondint/lastrop/BISLastAskPrcDay.xml&divisionId=MBIS01010010000000&divisionNm=%25EC%25B5%259C%25EC%25A2%2585%25ED%2598%25B8%25EA%25B0%2580%25EC%2588%2598%25EC%259D%25B5%25EB%25A5%25A0&tabIdx=1&w2xHome=/xml/&w2xDocumentRoot='
driver.get(url)
date=driver.find_element(By.ID,'srchDt_input')

# 채권 금리 정보 날짜 확인
eval_date=datetime.datetime.strptime(date.get_attribute('value'),'%Y-%m-%d').date()

elements = WebDriverWait(driver, 10).until(
    EC.visibility_of_all_elements_located((By.XPATH, '/html/body/div[1]/div[4]/div/div[1]/div/table/tbody'))
)

# 문장에서 '~'와 ' ~ ' 찾아서 삭제하는 함수
def symstrip(sentence):
    import re
    # 정규표현식을 사용하여 '~' 또는 ' ~ '을 포함하는 단어 찾기
    matches = re.findall(r'\b\w*\s*~\s*\w*\b', sentence)

    # '~' 또는 ' ~ '을 기준으로 왼쪽과 오른쪽을 분리하여 다시 조합하고, 원래 문장에 대체
    for match in matches:
        left, right = re.split(r'\s*~\s*', match)
        replacement = f"{left.strip()} {right.strip()}"
        sentence = re.sub(re.escape(match), replacement, sentence)

    return sentence


for element in elements:
    data_io=StringIO(symstrip(element.text))
# read_csv를 사용하여 데이터프레임 생성, sep='\t'로 탭 구분 명시
df = pd.read_csv(data_io, sep=" ", header=None)
df_new=df.iloc[:,[0,3]]
df_new.columns=['Btype','Yield']
df1=df_new.iloc[0:8,:]
df1=pd.concat([df_new.iloc[9,:].to_frame().T,df1],ignore_index=True)

# tenor 삽입
tenors=['03M','01Y','02Y','03Y','05Y','10Y','20Y','30Y','50Y']
df1['Mat']=tenors
M_Date=[eval_date+relativedelta(months=+3)]
years=[1,2,3,5,10,20,30,50]
for year in years:
    M_Date.append(eval_date+relativedelta(years=+year))
df1['M_Date']=M_Date

# 일자 간 차이 계산 (일(day) 단위로)
df1['Days_Difference'] = df1.M_Date.apply(lambda x: (x-eval_date).days)
df1['Coupons']=0.05 # 표면금리 설정 (예)
df1.Yield=df1.Yield.astype(float) # 칼럼 자료를 float로 변환
df1['Prices']=[99,96.5,96,95.5,95,94.5,90,80,70]
df1

,Btype,Yield,Mat,M_Date,Days_Difference,Coupons,Prices
0,통안증권(91일),3.447,03M,2024-06-25,92,0.05,99.0
1,국고채권(1년),3.325,01Y,2025-03-25,365,0.05,96.5
2,국고채권(2년),3.326,02Y,2026-03-25,730,0.05,96.0
3,국고채권(3년),3.276,03Y,2027-03-25,1095,0.05,95.5
4,국고채권(5년),3.312,05Y,2029-03-25,1826,0.05,95.0
5,국고채권(10년),3.358,10Y,2034-03-25,3652,0.05,94.5
6,국고채권(20년),3.338,20Y,2044-03-25,7305,0.05,90.0
7,국고채권(30년),3.254,30Y,2054-03-25,10957,0.05,80.0
8,국고채권(50년),3.239,50Y,2074-03-25,18262,0.05,70.0


In [ ]:
# 날짜 설정
edate=ql.Date(eval_date.day,eval_date.month,eval_date.year) # datetime의 날짜를 Quantlib 날짜로 변환
ql.Settings.instance().evaluationDate=edate # 계산일 설정

# Market convetions
calendar=ql.SouthKorea()
convention=ql.ModifiedFollowing
day_count=ql.ActualActual(ql.ActualActual.ISMA)
end_of_month=True
fixing_days=1
face_amount=100
coupon_frequency=ql.Period(ql.Semiannual)

# Construct Bond Helpers
bond_helpers=[]
quote=df1 # 채권 yield 정보 데이터프레임 연결
for p,c,m in zip(quote.Prices,quote.Coupons,quote.Days_Difference):
        termination_date=edate+ql.Period(m,ql.Days)
        schedule=ql.Schedule(edate,termination_date,coupon_frequency,calendar,convention,convention,ql.DateGeneration.Backward,end_of_month)
        bond_helper=ql.FixedRateBondHelper(ql.QuoteHandle(ql.SimpleQuote(p)),fixing_days,face_amount,schedule,[c],day_count,convention)
        bond_helpers.append(bond_helper)

# Build curve : 선형 보간법으로 커브 만들기
curve=ql.PiecewiseLinearZero(edate,bond_helpers,day_count)

In [ ]:
curve.zeroRate(edate, day_count, ql.Compounded, ql.Continuous).rate()

0.09297117557024137

In [ ]:
import matplotlib.pyplot as plt
# 시각화를 위해 날짜 범위 설정
start_date = edate
end_date = start_date + ql.Period(50, ql.Years) # 예시로 30년까지의 범위 설정

# 곡선 플로팅
dates = [start_date + ql.Period(n, ql.Months) for n in range(0, 12 * 50)] # 50년 동안의 월별 날짜 생성
rates = [curve.zeroRate(date, day_count, ql.Compounded, ql.Continuous).rate() for date in dates] # 각 날짜에 대한 zero rate 계산

# Date 객체를 문자열로 변환
date_str = ql.Date.to_date_string(dates)

plt.figure(figsize=(10, 6))
plt.plot(date_str, rates, label='Zero Rate Curve')
plt.title('Zero Rate Curve')
plt.xlabel('Date')
plt.ylabel('Zero Rate')
plt.grid(True)
plt.legend()
plt.show()

AttributeError: type object 'Date' has no attribute 'to_date_string'

In [ ]:
dates

[Date(25,3,2024),
 Date(25,4,2024),
 Date(25,5,2024),
 Date(25,6,2024),
 Date(25,7,2024),
 Date(25,8,2024),
 Date(25,9,2024),
 Date(25,10,2024),
 Date(25,11,2024),
 Date(25,12,2024),
 Date(25,1,2025),
 Date(25,2,2025),
 Date(25,3,2025),
 Date(25,4,2025),
 Date(25,5,2025),
 Date(25,6,2025),
 Date(25,7,2025),
 Date(25,8,2025),
 Date(25,9,2025),
 Date(25,10,2025),
 Date(25,11,2025),
 Date(25,12,2025),
 Date(25,1,2026),
 Date(25,2,2026),
 Date(25,3,2026),
 Date(25,4,2026),
 Date(25,5,2026),
 Date(25,6,2026),
 Date(25,7,2026),
 Date(25,8,2026),
 Date(25,9,2026),
 Date(25,10,2026),
 Date(25,11,2026),
 Date(25,12,2026),
 Date(25,1,2027),
 Date(25,2,2027),
 Date(25,3,2027),
 Date(25,4,2027),
 Date(25,5,2027),
 Date(25,6,2027),
 Date(25,7,2027),
 Date(25,8,2027),
 Date(25,9,2027),
 Date(25,10,2027),
 Date(25,11,2027),
 Date(25,12,2027),
 Date(25,1,2028),
 Date(25,2,2028),
 Date(25,3,2028),
 Date(25,4,2028),
 Date(25,5,2028),
 Date(25,6,2028),
 Date(25,7,2028),
 Date(25,8,2028),
 Date(25,9,2028)

In [ ]:
len(rates)

600